In [36]:
from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import StockLatestQuoteRequest, CryptoBarsRequest, StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data import CryptoDataStream, StockDataStream
from load_api_keys import load_alpaca_paper_trading_keys
import datetime
api_keys = load_alpaca_paper_trading_keys()

crypto_client = CryptoHistoricalDataClient()
stock_client = StockHistoricalDataClient(api_key=api_keys.api_key, secret_key = api_keys.secret_key)

Making requests for multiple stocks and getting their quotes currently

In [4]:
stocks = ['SPY', 'TLT']
latest_quotes_request =  StockLatestQuoteRequest(symbol_or_symbols=stocks)
quotes = stock_client.get_stock_latest_quote(latest_quotes_request)

In [6]:
tlt_quote = quotes['TLT']

In [9]:
tlt_quote.bid_size

13.0

In [10]:
type(tlt_quote)

alpaca.data.models.quotes.Quote

In [11]:
tlt_quote

{   'ask_exchange': 'V',
    'ask_price': 98.54,
    'ask_size': 13.0,
    'bid_exchange': 'V',
    'bid_price': 98.39,
    'bid_size': 13.0,
    'conditions': ['R'],
    'symbol': 'TLT',
    'tape': 'C',
    'timestamp': datetime.datetime(2022, 10, 14, 20, 20, 0, 500605, tzinfo=datetime.timezone.utc)}

We get a quote object back which we can get information from. In this quote, we can get: 
* ask prices and sizes
* bid prices and sizes
* timestamp of the last quote


#### Getting historical bars

In [15]:
bars_request = StockBarsRequest(
    symbol_or_symbols=stocks, 
    timeframe = TimeFrame.Day, 
    start = datetime.datetime(2022, 7, 1)
)

bars = stock_client.get_stock_bars(bars_request)

In [17]:
type(bars)

alpaca.data.models.bars.BarSet

In [25]:
spy_bars = bars.df.loc[('SPY', slice(None))]

In [21]:
bars['TLT'][0]

{   'close': 115.82,
    'high': 117.59,
    'low': 115.22,
    'open': 115.93,
    'symbol': 'TLT',
    'timestamp': datetime.datetime(2022, 7, 1, 4, 0, tzinfo=datetime.timezone.utc),
    'trade_count': 138117.0,
    'volume': 21856202.0,
    'vwap': 116.113939}

We get back a bars set object. The way to access a specific bar is to index the asset and then the number of the bar. The most recent bar is going to be at the end. There is also a dataframe representation as we see. We have a multi-index where the outermost index is the stock and the innermost index would be the timestamp which represents the close of business time. 

#### Real time market data

In [27]:
crypto_stream = CryptoDataStream(api_key=api_keys.api_key, secret_key=api_keys.secret_key)

In [30]:
async def quote_data_handler(data): 

    print(data)

crypto_stream.subscribe_quotes(quote_data_handler, 'BTC/USD')
# crypto_stream.run() Can't run this here now because of the event loop in the jupyter notebook

When you run this, you are going to get a bunch of quote objects which have the bid and ask prices from the exchange. We can play with this as usual but for now, probably going to just use the historical data. 


In [34]:
hour_bars_request = StockBarsRequest(
    symbol_or_symbols=stocks, 
    timeframe = TimeFrame.Hour, 
    start = datetime.datetime(2022, 7, 1)
)

hour_bars = stock_client.get_stock_bars(hour_bars_request)

In [38]:
thirty_minute_timeframe = TimeFrame(amount = 30, unit = TimeFrameUnit.Minute)

In [39]:
time_request = StockBarsRequest(
    symbol_or_symbols='SPY', 
    timeframe=thirty_minute_timeframe, 
    start = datetime.datetime(2022, 7, 1)
)

thirty_minute_bars = stock_client.get_stock_bars(time_request)

In [40]:
thirty_minute_bars.df

open    high     low   close   volume  \
symbol timestamp                                                            
SPY    2022-07-01 04:00:00+00:00  374.40  374.40  373.59  373.59   2124.0   
       2022-07-01 04:30:00+00:00  373.80  374.18  373.80  374.00    840.0   
       2022-07-01 05:00:00+00:00  373.85  373.85  373.85  373.85    452.0   
       2022-07-01 05:30:00+00:00  374.30  374.30  374.30  374.30    100.0   
       2022-07-01 07:00:00+00:00  373.50  373.50  373.50  373.50    158.0   
...                                  ...     ...     ...     ...      ...   
       2022-10-14 21:30:00+00:00  357.22  357.50  357.20  357.20  27498.0   
       2022-10-14 22:00:00+00:00  357.23  357.35  357.05  357.07  32190.0   
       2022-10-14 22:30:00+00:00  357.05  357.63  356.97  356.97  18606.0   
       2022-10-14 23:00:00+00:00  356.96  357.00  356.76  356.76  15210.0   
       2022-10-14 23:30:00+00:00  356.85  356.89  356.50  356.60  61017.0   

                                  trade_count        vwap  
symbol timestamp                                           
SPY    2022-07-01 04:00:00+00:00         14.0  373.989487  
       2022-07-01 04:30:00+00:00          5.0  373.902381  
       2022-07-01 05:00:00+00:00          2.0  373.850000  
       2022-07-01 05:30:00+00:00          1.0  374.300000  
       2022-07-01 07:00:00+00:00          8.0  373.511139  
...                                       ...         ...  
       2022-10-14 21:30:00+00:00        423.0  357.334160  
       2022-10-14 22:00:00+00:00        559.0  357.241808  
       2022-10-14 22:30:00+00:00        388.0  357.019851  
       2022-10-14 23:00:00+00:00        449.0  356.881290  
       2022-10-14 23:30:00+00:00        955.0  356.656834  

[2710 rows x 7 columns]